In [1]:
from sklearn import preprocessing

In [3]:
transfo = preprocessing.LabelEncoder()

In [8]:
transfo.fit(['a','b','r','c'])

LabelEncoder()

In [9]:
transfo.transform(['b','a','r','c'])

array([1, 0, 3, 2], dtype=int64)

In [11]:
salut={'r':'tru'}

In [17]:
salut.get('r',4)

'tru'

In [18]:
3+4

7

In [19]:
int(4.5)

4